In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import json

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [13]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-tinyllama-1.1b-16384-instructions')

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-tinyllama-1.1b-16384-instructions',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

In [4]:
def parse_llama_chat(messages, function_call = None):

    system = messages[0]['content']
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[1:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = [f'<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n\n']
    if function_call:
        fs = []
        for f in function_call:
            f = json.dumps(f, indent=4)
            fs.append(f)
        fs = '\n\n'.join(fs)
        texts.append(f'\n[FUNCTIONCALL]\n{fs}\n')
    for u, a in zip(users, assistants):
        texts.append(f'{u.strip()} [/INST] {a.strip()} </s><s>[INST] ')
    texts.append(f'{user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [5]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'kwsp tu apa'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [6]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
tokenizer.decode(r[0])

'<s> [INST] <<SYS>>\nawak adalah AI yang mampu jawab segala soalan\n<</SYS>>\n\nkwsp tu apa [/INST] KWSP bermaksud Kumpulan Wang Persaraan. </s>'

In [7]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'awat malaysia ada jabatan koko, malaysia bukan buat keluaq koko banyak pun'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [8]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

<s> [INST] <<SYS>>
awak adalah AI yang mampu jawab segala soalan
<</SYS>>

awat malaysia ada jabatan koko, malaysia bukan buat keluaq koko banyak pun [/INST] Jabatan ini bertanggungjawab untuk mengeluarkan dan mengagihkan produk koko ke pasaran tempatan dan antarabangsa. Mereka juga menyumbang kepada pembangunan industri koko dan memastikan penggunaan sumber asli yang bertanggungjawab. Selain itu, mereka menjalankan penyelidikan dan inovasi untuk meningkatkan proses pengeluaran dan meningkatkan daya saing produk koko. </s>


In [9]:
f2 = {
    'name': 'parse_entities',
    'description': 'extract entities from the text',
    'parameters': {
        'type': 'object',
        'properties': {
            'drink': {
                'type': 'string',
                'description': 'drink name',
            },
            'event': {
                'type': 'string',
                'description': 'event name',
            },
            'person_name': {
                'type': 'string',
                'description': 'person name',
            }
        },
        'required': [
            'drink',
            'event',
            'person_name'
        ]
    }
}

In [18]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'nama saya husein bin zolkepli, saya sekarang berada di putrajaya merdeka 2023 sambil minum teh o ais'}
]
prompt = parse_llama_chat(messages, function_call = [f2])
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

<s> [INST] <<SYS>>
awak adalah AI yang mampu jawab segala soalan
<</SYS>>


[FUNCTIONCALL]
{
    "name": "parse_entities",
    "description": "extract entities from the text",
    "parameters": {
        "type": "object",
        "properties": {
            "drink": {
                "type": "string",
                "description": "drink name"
            },
            "event": {
                "type": "string",
                "description": "event name"
            },
            "person_name": {
                "type": "string",
                "description": "person name"
            }
        },
        "required": [
            "drink",
            "event",
            "person_name"
        ]
    }
}
nama saya husein bin zolkepli, saya sekarang berada di putrajaya merdeka 2023 sambil minum teh o ais [/INST] <functioncall> {"name": "parse_entities", "arguments": '{"drink": "teh o ais", "event": "Merdeka 2023", "person_name": "Husein bin Zolkepli"}'} 


<functioncall> {"entities